In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from torch import nn
from sklearn import metrics

In [ ]:
class TabularDataset(torch.utils.data.Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return len(self.features)
    def __getitem__(self,item):
        return {
            'features': torch.tensor(self.features, dtype=torch.float32),
            'targets': torch.tensor(self.targets, dtype=torch.long)
        }

In [ ]:
class get_model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=1024, out_features=2048, bias=True)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(in_features=2048, out_features=1024, bias=True)
        self.drop = nn.Dropout(p=0.0, inplace=False)
        self.depth1 = nn.Linear(in_features=1024, out_features=526, bias=True)
        self.depth2 = nn.Linear(in_features=526, out_features=256, bias=True)
        self.out = nn.Linear(in_features=256, out_features=6, bias=True)
    def forward(self, features):
        x = self.fc1(features)
        x = self.act(x)
        x = self.fc2(x)
        x = self.drop(x)
        x = self.depth1(x)
        x = self.depth2(x)
        x = self.out(x)
        return x

In [ ]:
df = pd.read_csv('../input/feature-engineering-on-the-cover-type/TPS_dec_2021.csv')
cols = list(df.columns)
cols.remove('Cover_Type')
cols.remove('kfold')

In [ ]:
def train(model, dataloader, optimizer, device):
    model.train()
    
    for data in dataloader:
        features = data['features']
        targets = data['targets']
        
        features = features.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad() ## zeroed gradients
        outputs = model(features)
        self.loss = nn.CrossEntropyLoss()(torch.argmax(outputs, dim=1), targets)
        loss.backward()
        optimizer.step()
def validate(model, dataloader, device):
    model.eval()
    final_outputs = []
    final_targets = []
    
    with torch.no_grad():
        for data in dataloader:
            features = data['features']
            targets = data['targets']
            
            features = features.to(device)
            targets = targets.to(device)
            
            outputs = model(features)
            
            targets = (targets.detach().cpu().numpy()).tolist()
            outputs = (torch.argmax(outputs, dim=1).detach().cpu().numpy()).tolist()
            
            final_outputs.extend(outputs)
            final_targets.extend(targets)
            
    return final_outputs,final_targets
        

In [ ]:
epochs = 60
lr=5e-3
device = 'cuda'

for fold in range(10):
    model = get_model()
    model.to(device)
    df_train = df[df.kfold!=fold].reset_index(drop=True)
    df_valid = df[df.kfold==fold].reset_index(drop=True)
    
    df_train = df_train.drop(columns='kfold') 
    df_valid = df_valid.drop(columns='kfold')
    
    train_dataset = TabularDataset(df_train[cols].values, df_train.Cover_Type.values)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4096,
                                               shuffle=True, pin_memory=True)
    valid_dataset = TabularDataset(df_valid[cols].values, df_valid.Cover_Type.values)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=4096,
                                              shuffle=False, pin_memory=True)
    
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)
    print(f'you are in fold =====  fold {fold} ====')
    for epoch in range(epochs):
        
        train(model, train_loader, optimizer, device)
        final_outputs,final_targets = validate(model, valid_loader, device)
        
        acc = metrics.accuracy_score(final_targets,final_outputs)
        
        print(f'you are in epoch =====  epoch {epoch} ====')
        print(f'valid Accuracy =====  accuracy {acc} ====')
        
